In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
# vedeere DistanceMetric sklearn per le misure di distanza
# vedere il parametro metric, che permette di definire una matrice di distanze

db = pd.read_csv('dataset/db_modelli.csv').drop(columns = 'Unnamed: 0')
# codifica
db['conc1_mean'] = np.where(db['conc1_mean'].values > 1, 1, 0)



In [2]:
db = db.drop(columns = 'test_cas')
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23332 entries, 0 to 23331
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            23332 non-null  object 
 1   conc1_type         23332 non-null  object 
 2   exposure_type      23332 non-null  object 
 3   obs_duration_mean  23332 non-null  float64
 4   conc1_mean         23332 non-null  int32  
 5   atom_number        23332 non-null  float64
 6   alone_atom_number  23332 non-null  int64  
 7   bonds_number       23332 non-null  float64
 8   doubleBond         23332 non-null  int64  
 9   tripleBond         23332 non-null  int64  
 10  ring_number        23332 non-null  float64
 11  Mol                23332 non-null  float64
 12  MorganDensity      23332 non-null  float64
 13  LogP               23332 non-null  float64
 14  class              23332 non-null  object 
 15  tax_order          23332 non-null  object 
 16  family             233

In [3]:
lst = list()
for col in db.columns:
    if db[col].dtypes == 'object':
        lst.append(col)

print(lst)

['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']


In [4]:
encoder = OrdinalEncoder(dtype = int)

encoder.fit(db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])

db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']] = encoder.transform(
    db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])+1

db

,species,conc1_type,exposure_type,obs_duration_mean,conc1_mean,atom_number,alone_atom_number,bonds_number,doubleBond,tripleBond,ring_number,Mol,MorganDensity,LogP,class,tax_order,family,genus,oh_count
0,379,3,3,48.0,1,0.317908,2,0.488106,1,0,1.0,0.535725,1.300000,2.2482,1,10,35,80,0
1,379,3,3,96.0,1,0.317908,2,0.488106,1,0,1.0,0.535725,1.300000,2.2482,1,10,35,80,0
2,379,3,9,96.0,1,0.317908,2,0.488106,1,0,1.0,0.510371,1.300000,1.1770,1,10,35,80,0
3,218,3,1,48.0,1,0.317908,2,0.488106,1,0,1.0,0.510371,1.300000,1.1770,1,10,35,134,0
4,237,1,9,24.0,1,0.317908,2,0.488106,1,0,1.0,0.510371,1.300000,1.1770,1,6,2,182,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23327,199,3,9,48.0,1,0.547662,0,0.678290,2,0,2.0,0.749544,0.703704,5.5670,1,10,35,266,2
23328,199,3,9,96.0,1,0.547662,0,0.678290,2,0,2.0,0.749544,0.703704,5.5670,1,10,35,266,2
23329,274,3,8,24.0,0,0.547662,0,0.678290,2,0,2.0,0.749544,0.703704,5.5670,1,23,77,176,2
23330,274,3,8,48.0,0,0.547662,0,0.678290,2,0,2.0,0.749544,0.703704,5.5670,1,23,77,176,2


In [5]:
y = db['conc1_mean'].values
X = db.drop(columns = 'conc1_mean').values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)


In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.7322077922077922

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Create new KNN object
knn_2 = KNeighborsClassifier()

#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=3)

#Fit the model
best_model = clf.fit(X_train, y_train)

#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

KeyboardInterrupt: 